In [19]:
from pathlib import Path

import pandas as pd
from collections import defaultdict

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [20]:
dir = "/Volumes/USB SanDisk 3.2Gen1 Media/a4d/a4dphase2_upload/output/logs"

In [21]:
files = list(Path(dir).glob("*.log"))
len(files)

43

In [22]:
data = defaultdict(list)
for file in files:
    with open(file, "r", encoding="utf-8") as fstream:
        for line in fstream.read().splitlines():
            error = None
            warn = None
            if "ERROR" in line:
                error = line.split("\t")[-1]
                
            if "WARN" in line:
                warn = line.split("\t")[-1]
            
            if error or warn: 
                data["file"].append(file.name)
                data["error"].append(error)
                data["warning"].append(warn)

In [23]:
df = pd.DataFrame(data)

In [24]:
df

,file,error,warning
0,2021_Putrajaya Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: recommendedtestingfrequencyperday,hospitalisationduetodiabetesemergencyorglucosecontrolreason,hospitalisationduetodiabetesemergencyorglucosecontroldate,complicationscreeningselect,patientobservations,observationcategoryselect,suggestionsforpotentialadditionalsupportorinterventionfroma4degcomplicationsorfamilysupport."
1,2021_Putrajaya Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: recommendedtestingfrequencyperday,hospitalisationduetodiabetesemergencyorglucosecontrolreason,hospitalisationduetodiabetesemergencyorglucosecontroldate,complicationscreeningselectfordropdown,complicationscreening,otherpatientobservations,observationcategoryselect,suggestionsforpotentialadditionalsupportorinterventionfroma4degcomplicationsorfamilysupport."
2,2021_Putrajaya Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: virtualremotecontactyn,virtualremotecontactdate,recommendedtestingfrequencyperday,hospitalisationduetodiabetesemergencyorglucosecontrolreason,hospitalisationduetodiabetesemergencyorglucosecontroldate,complicationscreeningselectfordropdown,complicationscreening,otherpatientobservations,observationcategoryselect,suggestionsforpotentialadditionalsupportorinterventionfroma4degcomplicationsorfamilysupport."
3,2021_Putrajaya Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: virtualremotecontactyn,virtualremotecontactdate,recommendedtestingfrequencyperday,currentmonthhospitalisationduetodiabetesemergencyorglucosecontrolreason,currentmonthhospitalisationduetodiabetesemergencyorglucosecontroldate,complicationscreeningcurrentmonthtestingselectfordropdown,complicationscreeningcurrentmonthtesting,otherpatientobservations,observationcategoryselect,suggestionsforpotentialadditionalsupportorinterventionfroma4degcomplicationsorfamilysupport."
4,2021_Putrajaya Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: virtualremotecontactyn,virtualremotecontactdate,recommendedtestingfrequencyperday,currentmonthhospitalisationduetodiabetesemergencyorglucosecontrolreason,currentmonthhospitalisationduetodiabetesemergencyorglucosecontroldate,complicationscreeningcurrentmonthtestingselectfordropdown,complicationscreeningcurrentmonthtesting,otherpatientobservations,observationcategoryselect,suggestionsforpotentialadditionalsupportorinterventionfroma4degcomplicationsorfamilysupport,meter."
...,...,...,...
266,2022_Sultanah Bahiyah A4D Tracker_DC_patient.log,None,"Non-matching column names found: updatedhba1chba1cdate,updatedfbgfbgmmoll,updatedfbgfbgdate."
267,2022_Sultanah Bahiyah A4D Tracker_DC_patient.log,None,"Non-matching column names found: updatedhba1chba1cdate,updatedfbgfbgmmoll,updatedfbgfbgdate."
268,2022_Sultanah Bahiyah A4D Tracker_DC_patient.log,None,Non-matching column names found: newmeter.
269,2022_Sultanah Bahiyah A4D Tracker_DC_patient.log,None,Non-matching column names found: newmeter.


In [16]:
df.to_csv(Path(dir) / "error_report.csv", index=False)

In [25]:
df.file.nunique()

20

## Error

In [26]:
df[~df.error.isna()]

,file,error,warning
28,2018_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2018_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
29,2018_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2019_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
30,2018_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2020_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
31,2018_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2021_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
43,2018_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2022_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
76,2022_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2022_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
92,2021_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2021_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
104,2021_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2022_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
144,2019_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2019_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None
145,2019_Lao Friends Hospital for Children A4D Tracker_DC_patient.log,Could not process 2020_Lao Friends Hospital for Children A4D Tracker_DC. Error = Error: row_min < row_max is not TRUE .,None


In [9]:
# how many files could not be processed in %?
len(df[(~df.error.isna()) & (df.error.str.contains("Could not process"))]) / len(files) * 100
                                                        

35.8974358974359

In [10]:
# missing patient id
df[(~df.error.isna()) & (df.error.str.contains("patient_id"))].file.tolist()

[]

In [11]:
# month list is empty?
print("\n".join(l.split(".")[0] for l in df[(~df.error.isna()) & (df.error.str.contains("month_list"))].file.tolist()))

In [12]:
# cannot find patient data
for year in range(2017,2023):
    subdf = df[(~df.error.isna()) & (df.error.str.contains("readxl::cell_limits"))]
    print(subdf[subdf.file.str.startswith(str(year))].file.tolist())

[]
[]
[]
[]
[]
[]


In [13]:
# check problem with num_na_rows
subdf = df[(~df.error.isna()) & (df.error.str.contains("num_na_rows"))]
subdf

,file,error,warning


In [14]:
# check start_df_msd:end_df_msd
subdf = df[(~df.error.isna()) & (df.error.str.contains("start_df_msd"))]
subdf.file.tolist()

[]

# Warnings

In [18]:
df[~df.warning.isna()]

,file,error,warning
0,2022_Putrajaya Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: lostpatientssummarylostdate,lostpatientssummarystatusout."
1,2020_Sarawak General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: updatedhba1chba1cdate,updatedfbgfbgmmoll,updatedfbgfbgdate."
2,2020_Sarawak General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: updatedhba1chba1cdate,updatedfbgfbgmmoll,updatedfbgfbgdate."
3,2020_Sarawak General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: updatedhba1chba1cdate,updatedfbgfbgmmoll,updatedfbgfbgdate."
4,2020_Sarawak General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: updatedhba1chba1cdate,updatedfbgfbgmmoll,updatedfbgfbgdate."
...,...,...,...
239,2020_Penang General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: educationvocation,admissiontohospitaldatemmmyy,remarkscomplicationsfamilysupport."
240,2020_Penang General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: educationvocation,admissiontohospitaldatemmyy,remarkscomplicationsfamilysupport."
241,2020_Penang General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: educationvocation,admissiontohospitaldatemmyy,remarkscomplicationsfamilysupport."
242,2020_Penang General Hospital A4D Tracker_DC_patient.log,None,"Non-matching column names found: educationvocation,admissiontohospitaldatemmyy,remarkscomplicationsfamilysupport."


In [15]:
missed_names = df[(~df.warning.isna()) & (df.warning.str.startswith("Non-matching column names found"))].warning.str.split(":",expand=True)[1].drop_duplicates()

In [16]:
names = set()
for row in missed_names:
    names.update(w.strip(".") for w in row.split(","))

In [17]:
names

{' complicationscreeningdropdown',
 ' currentmonthcomplicationscreeningdropdown',
 ' educationvocation',
 ' hospitalisationduetodiabetesemergencydate',
 ' lostpatientssummarylostdate',
 ' newmeter',
 ' recommendedtestingfrequencyperday',
 ' updatedhba1chba1cdate',
 ' virtualremotecontactyn',
 'admissiontohospitaldatemmmyy',
 'admissiontohospitaldatemmyy',
 'complicationscreening',
 'complicationscreeningcurrentmonthtesting',
 'complicationscreeningcurrentmonthtestingselectfordropdown',
 'complicationscreeningselect',
 'complicationscreeningselectfordropdown',
 'currentmonthcomplicationscreening',
 'currentmonthhospitalisationduetodiabetesemergencyorglucosecontroldate',
 'currentmonthhospitalisationduetodiabetesemergencyorglucosecontrolreason',
 'hospitalisationduetodiabetesemergencyorglucosecontroldate',
 'hospitalisationduetodiabetesemergencyorglucosecontrolreason',
 'lostpatientssummarystatusout',
 'meter',
 'observationcategoryselect',
 'otherpatientobservations',
 'patientobservati